# 1. 슬라이딩 윈도우(Sliding Window)를 활용한 시계열 데이터 → 학습용 샘플로 만들기

### 코드에 대한 상세 설명

1.  **하이퍼파라미터 설정:**
    *   `WINDOW_SIZE`: 모델 성능에 큰 영향을 주는 중요한 파라미터입니다. 문제의 특성에 맞게 조절이 필요합니다. (예: 7로 설정하면, 과거 7개 이벤트를 보고 8번째 이벤트를 예측)
    *   `INPUT_FEATURES`: 모델의 입력으로 사용할, 수치로 변환된 컬럼들만 선택합니다.
    *   `TARGET_COLUMNS`: 모델이 맞춰야 할 정답입니다. `event_type_id` 외에 `location_id_id` 등을 추가하여 다중 예측 문제로 만들 수도 있습니다.

2.  **`create_sliding_window_dataset` 함수:**
    *   `df.groupby('epc_code')`: **가장 중요한 부분입니다.** 각 시퀀스(`epc_code`)별로 따로 슬라이딩 윈도우를 적용해야, 한 시퀀스의 끝과 다른 시퀀스의 시작이 섞이는 것을 방지할 수 있습니다.
    *   `tqdm`: 데이터가 많을 때 처리 시간 예측을 위해 진행 바(progress bar)를 보여주는 유용한 라이브러리입니다.
    *   `features = group[INPUT_FEATURES].values`: pandas 데이터프레임을 NumPy 배열로 변환합니다. 이게 계산 속도가 훨씬 빠릅니다.
    *   `for i in range(...)`: 루프를 돌면서 윈도우를 한 칸씩( `step_size` 만큼) 옆으로 이동시킵니다.
    *   `X_data.append(...)`, `y_data.append(...)`: 생성된 샘플들을 파이썬 리스트에 차곡차곡 쌓습니다.
    *   `np.array()`: 모든 처리가 끝난 후, 리스트를 최종적으로 NumPy 배열로 변환합니다.

3.  **저장 (`np.savez_compressed`):**
    *   생성된 `X`와 `y` 배열을 하나의 압축 파일(`.npz`)로 저장합니다.
    *   나중에 모델 학습 코드에서는 `data = np.load('train_dataset.npz')`, `X_train = data['X']`, `y_train = data['y']` 와 같이 쉽게 불러올 수 있습니다.

이제 이 코드를 실행하시면 `train_dataset.npz`와 `test_dataset.npz` 파일이 생성되며, 이 파일들이 바로 딥러닝 모델을 학습시킬 준비가 완료된 최종 입력 데이터입니다.

In [ ]:
# ===== 셀 1: (LSTM용) 슬라이딩 윈도우 데이터셋 생성 =====
# 이 셀은 Window Size를 바꾸지 않는 한, 한 번만 실행하면 됩니다.
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path

OUTDIR = Path("preprocessed")
# --- 1. 하이퍼파라미터 설정 ---
# 모델이 몇 개의 과거 이벤트를 보고 다음을 예측할지 결정합니다.
WINDOW_SIZE = 3  # 예: 과거 7개 이벤트를 보고
STEP_SIZE = 1    # 윈도우를 한 칸씩 이동

# 모델의 입력으로 사용할 피처(컬럼) 목록을 정의합니다.
# 원본 ID, 시간 정보 등은 보통 제외하고, 인코딩/스케일링된 값만 사용합니다.
INPUT_FEATURES = [
    'event_type_id',
    'location_id_id',
    'delta_t_sec',  # 스케일링 완료된 시간 차이
    'hour_sin',     # 주기성 피처
    'hour_cos'      # 주기성 피처
]

# 모델이 예측해야 할 정답(Target) 컬럼을 정의합니다.
# 가장 일반적인 목표는 '다음 이벤트 타입'을 예측하는 것입니다.
TARGET_COLUMNS = ['event_type_id']


# --- 2. 데이터셋 생성을 위한 함수 정의 ---
def create_sliding_window_dataset(df: pd.DataFrame, window_size: int, step_size: int):
    X_data, y_data = [], []
    
    # tqdm을 사용해 epc_code 그룹별로 루프를 실행하며 진행 상황을 표시합니다.
    grouped = df.groupby('epc_code')
    for _, group in tqdm(grouped, desc="Processing sequences"):
        # 각 시퀀스를 feature와 target으로 분리합니다.
        features = group[INPUT_FEATURES].values
        targets = group[TARGET_COLUMNS].values
        
        # 시퀀스 길이가 윈도우 크기보다 짧으면 샘플을 만들 수 없으므로 건너뜁니다.
        # (예: 윈도우가 7이면, 최소 8개의 이벤트가 있어야 X, y 쌍 1개를 만듦)
        if len(group) < window_size + 1:
            continue
            
        # 슬라이딩 윈도우를 적용하여 X, y 쌍을 생성합니다.
        for i in range(0, len(group) - window_size, step_size):
            window_end = i + window_size
            
            # 입력 시퀀스 (X)
            input_window = features[i:window_end]
            X_data.append(input_window)
            
            # 정답 (y) - 윈도우 바로 다음 이벤트
            target_event = targets[window_end]
            y_data.append(target_event)
            
    # 파이썬 리스트를 딥러닝에 적합한 NumPy 배열로 변환합니다.
    return np.array(X_data), np.array(y_data)

# --- 3. Train/Test 데이터셋 생성 및 저장 ---
def process_and_save_dataset(name: str):
    print(f"\n--- {name.upper()} 데이터셋 생성 시작 ---")
    parquet_path = OUTDIR / f"{name}_final.parquet"
    csv_path = OUTDIR / f"{name}_final.csv"

    # parquet 파일이 있으면 읽고, 없으면 csv 파일을 읽습니다.
    if os.path.exists(parquet_path):
        df = pd.read_parquet(parquet_path)
    else:
        df = pd.read_csv(csv_path, parse_dates=['event_time'])

    # 슬라이딩 윈도우 함수를 호출합니다.
    X, y = create_sliding_window_dataset(df, WINDOW_SIZE, STEP_SIZE)

    # 생성된 데이터의 shape(모양)을 출력합니다.
    # X shape: (총 샘플 수, 윈도우 크기, 피처 개수)
    # y shape: (총 샘플 수, 타겟 개수)
    print(f"생성된 {name} 데이터 shape: X={X.shape}, y={y.shape}")

    # NumPy 포맷으로 최종 데이터를 저장합니다.
    # .npz는 여러 배열을 한 파일에 압축 저장하는 효율적인 포맷입니다.
    np.savez_compressed(
        OUTDIR / f"{name}_dataset_ws3.npz",
        X=X,
        y=y
    )
    print(f"[OK] {name.upper()} 데이터셋 저장 완료: {OUTDIR / f'{name}_dataset_ws3.npz'}")

process_and_save_dataset("train")
process_and_save_dataset("test")

print("\n모든 작업이 완료되었습니다. 이제 .npz 파일을 이용해 모델 학습을 시작할 수 있습니다.")

# 8. LSTM 예측 모델 훈련 및 저장

In [ ]:
# ===== 셀 2: 모델 학습 및 앙상블 =====
import numpy as np
import json
import matplotlib.pyplot as plt
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


# 이전 셀에서 정의했던 OUTDIR 변수를 다시 선언해줍니다.
# --- [수정 1] 변수 정의 ---
# --- 설정 및 어휘집 파일 로드 ---
OUTDIR = Path("preprocessed")
WINDOW_SIZE = 3

print("[INFO] 어휘집 파일을 로드합니다...")
with open(OUTDIR / "event_type.vocab.json", "r") as f: vocab_event = json.load(f)
VOCAB_SIZE_EVENT = len(vocab_event['id2token'])
data = np.load(OUTDIR / f"train_dataset_ws{WINDOW_SIZE}.npz")
X_train, y_train = data['X'], data['y']
# -----------------------------

# --- GPU 설정 ---
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try: tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e: print(e)


# --- 2. 모델 구조 정의 (재사용 가능하도록 함수로 만듦) ---
# 어휘집 크기(예측할 클래스 개수)를 가져옵니다.
# (실제로는 어휘집 json 파일을 로드해서 클래스 개수를 정확히 파악해야 합니다)
VOCAB_SIZE_EVENT = len(vocab_event['id2token'])

def build_regularized_model(input_shape, num_classes):
    model = Sequential([
        LSTM(64, input_shape=input_shape, return_sequences=True),
        Dropout(0.2),
        LSTM(32),
        Dropout(0.2),
        Dense(num_classes, activation='softmax') # 다중 분류를 위한 softmax
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# --- 3. 각 모델 개별 학습 ---
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model = build_regularized_model(X_train.shape[1:], VOCAB_SIZE_EVENT)
history = model.fit(X_train, y_train, epochs=20, batch_size=256, validation_split=0.2, callbacks=[early_stopping])
model.save(OUTDIR / "epc_predictor_final_model.keras")
print(f"\n[SUCCESS] 훈련된 예측 모델 저장 완료: {OUTDIR / 'epc_predictor_final_model.keras'}")

def plot_history(history_data, model_name):
    plt.figure(figsize=(12, 5))
    
    # 한글 폰트 설정
    plt.rcParams['font.family'] = 'D2Coding'
    plt.rcParams['axes.unicode_minus'] = False

    # 손실(Loss) 그래프
    plt.subplot(1, 2, 1)
    plt.plot(history_data['loss'], label='훈련 손실 (Train Loss)', marker='o')
    plt.plot(history_data['val_loss'], label='검증 손실 (Validation Loss)', marker='o')
    plt.title(f'{model_name} - 손실 그래프', fontsize=14)
    plt.xlabel('에포크 (Epoch)')
    plt.ylabel('손실 (Loss)')
    plt.legend()
    plt.grid(True)
    
    # 정확도(Accuracy) 그래프
    plt.subplot(1, 2, 2)
    plt.plot(history_data['accuracy'], label='훈련 정확도 (Train Accuracy)', marker='o')
    plt.plot(history_data['val_accuracy'], label='검증 정확도 (Validation Accuracy)', marker='o')
    plt.title(f'{model_name} - 정확도 그래프', fontsize=14)
    plt.xlabel('에포크 (Epoch)')
    plt.ylabel('정확도 (Accuracy)')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()